In [ ]:
import sys
import os 

import functions
from functions import *

# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
bounds = unpickle_data("../data/interim/bounds.pickle")
myorigins = get_origins(
    places = bounds,
    population_file = "../data/interim/pop_phl_national_capital_region.tif",
    window_size = 50,
    use_pickle = False,
    do_pickle_result = True
)



In [ ]:
# Optional - Snap Origins to Roads
#myorigins = origins_snap_osm(myorigins, "./data/networks/ncr_unclean_1.pickle")

In [ ]:
myorigins.head()

In [ ]:
# IF Snapped:
#visual= plot_df_single(myorigins, 'geometry', color = 'blue', alpha = 0.8, hover_cols=['tot_pop']) \
#* plot_df_single(myorigins, 'center_geom', color='red', alpha = 0.4, hover_cols=['tot_pop']) \
#* plot_df_single(myorigins, 'geo_grid', color = 'cyan', alpha = 0.2, hover_cols=['tot_pop'])

# IF Not Snapped
visual= plot_df_single(myorigins, 'geometry', color = 'blue', alpha = 0.8, hover_cols=['tot_pop']) \
* plot_df_single(myorigins, 'geo_grid', color = 'cyan', alpha = 0.2, hover_cols=['tot_pop'])


visual

In [ ]:
#Get Destinations
mydest = get_destinations(
    places = ["NCR,Philippines"],
    tags = {'amenity':['hospital','clinic']}
)
mydest.head(60)

In [ ]:
plot_df_single(mydest, color='blue', hover_cols = ['amenity']) 

In [ ]:
origins_walking = get_travel_times_mapbox(origins = myorigins,
                                  destinations = mydest,
                                  mode = 'walking',
                                  d_name = 'hosp_or_clinic',
                                  dest_id_col = "osmid",
                                  n_keep = 2,
                                  num_retries = 2,
                                  starting_token_index = 0,
                                  use_pickle=False,
                                  do_pickle_result=True)

origins_walking = origins_walking.loc[origins_walking['mins_to_hosp_or_clinic'] > -1, :]

In [ ]:
origins_driving = get_travel_times_mapbox(origins = myorigins,
                                  destinations = mydest,
                                  mode = 'driving',
                                  d_name = 'hosp_or_clinic',
                                  dest_id_col = "osmid",
                                  n_keep = 2,
                                  num_retries = 2,
                                  starting_token_index = 0,
                                  use_pickle=False,
                                  do_pickle_result=True)

origins_driving = origins_driving.loc[origins_driving['mins_to_hosp_or_clinic'] > -1, :]

In [ ]:
#This is acting up driving-traffic causing a 422 error
origins_traffic = get_travel_times_mapbox(origins = myorigins,
                                  destinations = mydest,
                                  mode = 'driving-traffic',
                                  d_name = 'hosp_or_clinic',
                                  dest_id_col = "osmid",
                                  n_keep = 2,
                                  num_retries = 2,
                                  starting_token_index = 0,
                                  use_pickle=False,
                                  do_pickle_result=True)

origins_traffic = origins_traffic.loc[origins_traffic['mins_to_hosp_or_clinic'] > -1, :]

In [ ]:
origins_cycling = get_travel_times_mapbox(origins = myorigins,
                                  destinations = mydest,
                                  mode = 'cycling',
                                  d_name = 'hosp_or_clinic',
                                  dest_id_col = "osmid",
                                  n_keep = 2,
                                  num_retries = 2,
                                  starting_token_index = 0,
                                  use_pickle=False,
                                  do_pickle_result=True)

origins_cycling = origins_cycling.loc[origins_cycling['mins_to_hosp_or_clinic'] > -1, :]

In [ ]:
hrs_to_hosp_or_clinic_walking=mapbox_analysis("walking",origins_walking, mydest)
hrs_to_hosp_or_clinic_driving=mapbox_analysis("driving",origins_driving, mydest)
hrs_to_hosp_or_clinic_traffic=mapbox_analysis("driving-traffic",origins_traffic, mydest)
hrs_to_hosp_or_clinic_cycling=mapbox_analysis("cycling",origins_driving, mydest)

In [ ]:
plot_access(origins_walking,mydest,tmax=1)

In [ ]:
plot_access(origins_driving,mydest,tmax=1)

In [ ]:
plot_access(origins_traffic,mydest,tmax=1)

In [ ]:
plot_access(origins_cycling,mydest,tmax=1)

In [ ]:
biplot(origins=origins_driving,destinations=mydest,t_max=1, mode="driving", xlim=(0,3))
biplot(origins=origins_traffic,destinations=mydest,t_max=1, mode="traffic", xlim=(0,3))
biplot(origins=origins_walking,destinations=mydest,t_max=1, mode="walking", xlim=(0,3))
biplot(origins=origins_cycling,destinations=mydest,t_max=1, mode="cycling", xlim=(0,3))